# Submodule 4: Comparative genomics analysis

In this submodule, you will begin with a directory of proteomes from de novo assembled and annotated genomes. 

A single bacterial genomes was analyzed in Submodules 1 & 2 and we automated the process on many genomes in Submodule 3 to procude a total of XX genome sequences.

We will add to this dataset, reference genomes publicly available from NCBI. These genome sequences are curated for quality and provide gene accessions with curated functional information. These datasets are crucial for providing context to our new dataset.


### Learning Objectives

- **Access Genome Datasets from the NCBI**:<br>
  Participants will use command line tools to acces publicly accessible genomes for use in a comparative genomics analysis.
    
- **Perform and Visualize Comparative Genomics Analyses**:<br>
  Gain proficiency in using comparative genomics tools and visuazing their outputs. Participants will use these outputs to identify patterns across genomes and develop hypotheses about genome relatedness, gene loss or gain events, and gene duplications.

- **Create and Understand Phylogeny Trees**:<br>
  Use both ortholog groups and average nucleotide identity in genes to create phylogeny trees. Use phylogeny trees to gain an understanding of the see species relatedness and to understand taxonomic groupings.

## **Install required software**

A few more tools are required for Submodule 4; OrthoFinder, UpSet plot, and fastANI. As with submodule 1, we will install these tools using __[Conda](https://docs.conda.io/en/latest/)__.

Each piece of software, along with links to publications and documentation, will be described in turn. Below is a brief summary of these tools.

### List of software
| **Tool**       | **Description**                                                                                                                                                           |
|:---------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **OrthoFinder**      | Finds orthogroups and orthologs, infers rooted gene trees for all orthogroups and identifies all of the gene duplication events in those gene trees.                         |
| **UpSet Plots**      | UpSet plots are a data visualization method for showing set data with more than three intersecting sets.                                                  |
| **fastANI**        | Developed for fast alignment-free computation of whole-genome Average Nucleotide Identity (ANI). ANI is defined as mean nucleotide identity of orthologous gene pairs shared between two microbial genomes.                                         |

## Starting Data

This submodule begins with a directory of proteomes in FAA (FASTA amino acid) format. This module is designd to work with data produced from ssubmodule 3, but replace the FAA files within the working directory *proteomes* or add additional FAA files as neeeded.

In [ ]:
%%bash

ls proteomes/




In [ ]:
#install the required packages
import requests
import json
import ipywidgets as widgets
from IPython.display import display
import random
print("done installing required packages")

#install the module quiz_module.py
##from quiz_module import run_quiz
from quiz_module import run_quiz
print("done installing quiz_module")

In [ ]:
#This randomizes the order of the possible answers.
##import_type should be one of two str values: 'json' or 'url'
##import_path here defines the json filepath
run_quiz(import_type="json", import_path="questions/1-1.json", instant_feedback=False, shuffle_questions=False, shuffle_answers=True)

## Process 1: **Contiguity** assessment using QUAST
- Program: **QUAST (Quality Assessment Tool for Genome Assemblies)**
- Citation: *Gurevich, A., Saveliev, V., Vyahhi, N., & Tesler, G. (2013). QUAST: quality assessment tool for genome assemblies. Bioinformatics, 29(8), 1072-1075.*
- Manual: https://github.com/ablab/quast

QUAST is a tool used to evaluate and compare the quality of genome assemblies by providing metrics such as N50, number of contigs, genome length, and misassemblies. Did you get one contig representing your entire genome? Or did you get thousands of contigs representing a highly fragmented genome?

QUAST has many functionalities which we will not explore in this tutorial, I encourage you to explore these, for now we are going to use it in its simplest form. The program parses the genome FASTA file and records statsitics about each contig, the length, GC content etc. This type of information is something you would typically provide in a publication or as a way to assess different assemblers/options you may use. 

The **input** to the program is the genome assembly **FASTA** and the output are various tables and an html/pdf you can export and view.

In [ ]:
%%bash

ete2

In [ ]:
# Phylogenetic tree

Phylo
ETE toolkit
ToyTree

https://github.com/etetoolkit/ete

http://etetoolkit.org/ipython_notebook/
 ETE Toolkit - Visualization and analyses using Ipython Notebooks 
The ETE toolkit - Ipython notebook integration

https://toytree.readthedocs.io/en/latest/

#South Dokota is doing one


